### The Lifecycle Configuration Script downloads the jar file to the jdbc directory under home directory (~)

In [ ]:
jar_dir <- "~/SageMaker/jdbc"
jar_file <- list.files(jar_dir, pattern = "Athena")
jar_dir_file <- file.path(jar_dir,jar_file)

### Replace the s3_bucket with the value of *SageMakerRS3BucketName* in the CloudFormation Outputs

* For example:-
 * *s3_bucket <- "blog-sagemaker-r-sagemakerrdatabucket-xxxxxxxxxxxxx"*

In [ ]:
s3_bucket <- "<value of SageMakerRS3BucketName in the CloudFormation Outputs>"

In [ ]:
region <- system("curl -s http://169.254.169.254/latest/dynamic/instance-identity/document | jq .region -r",intern=TRUE)
jdbc_string <- paste("jdbc:awsathena://AwsRegion=",region,";S3OutputLocation=s3://",s3_bucket,"/;AwsCredentialsProviderClass=com.simba.athena.amazonaws.auth.DefaultAWSCredentialsProviderChain",sep="")

In [ ]:
library(DBI)
library(rJava)
library(RJDBC)
options( java.parameters = "-Xmx8g" )
drv <- JDBC("com.simba.athena.jdbc.Driver",jar_dir_file,identifier.quote="`")
conn <- dbConnect(drv, jdbc_string)

In [ ]:
show_databases <- dbGetQuery(conn, "show databases")
show_databases

In [ ]:
dbListObjects(conn)

In [ ]:
fetch <- dbGetQuery(conn,"SELECT os, COUNT(*) count FROM mydatabase.cloudfront_logs WHERE date BETWEEN date '2014-07-05' AND date '2014-08-05' GROUP BY os;")

In [ ]:
fetch